In [1]:
! pip install pyspark
from pyspark.sql import *
spark = SparkSession.builder \
   .master("local[*]") \
   .appName("ImageClassification") \
   .config("spark.executor.memory", "16gb") \
   .config("spark.driver.memory", "16G") \
   .config("spark.driver.offHeap.enabled", "true") \
   .config("spark.driver.offHeap.size", "16G") \
   .config("spark.executor.maxResultSize", "16gb") \
   .getOrCreate()

     |████████████████████████████████| 212.3MB 26kB/s 
     |████████████████████████████████| 204kB 51.2MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=bb1e394242ea448013d7168d5805ff295c03f58d133f874e757be3a5ef56b2fa
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [3]:
# innstall java
!apt-get update

!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"
# install findspark using pip
!pip install -q findspark

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:13 https://

In [4]:
os.environ["SPARK_HOME"]
import findspark
findspark.init()

In [5]:
# Check the pyspark version
import pyspark
print(pyspark.__version__)

3.1.1


In [6]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [7]:
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=ImageClassification>

In [8]:

%tensorflow_version 1.x

TensorFlow 1.x selected.


In [9]:

import pyspark.sql.functions as f
! pip install sparkdl
! pip install tensorframes
! pip install kafka-python
! pip install tensorflowonspark
! pip install pillow
! pip install nose 
! pip install h5py
! pip install py4j


     |████████████████████████████████| 102kB 6.9MB/s 
     |████████████████████████████████| 256kB 13.3MB/s 
     |████████████████████████████████| 51kB 5.2MB/s 
     |████████████████████████████████| 163kB 10.0MB/s 


In [10]:
import pyspark # pyspark library
import sparkdl as dl # deep learning library : tensorflow backend 
from keras.applications import InceptionV3 # transfer learning using pyspark

Using TensorFlow backend.


In [11]:
import os

SUBMIT_ARGS = "--packages databricks:spark-deep-learning:1.5.0-spark2.4-s_2.11 pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = SUBMIT_ARGS


import sys, glob, os
sys.path.extend(glob.glob(os.path.join(os.path.expanduser("~"), ".ivy2/jars/*.jar")))
from pyspark.ml.image import ImageSchema
from pyspark.sql.functions import lit

In [12]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()


In [17]:
import sys, glob, os
sys.path.extend(glob.glob(os.path.join(os.path.expanduser("~"), ".ivy2/jars/*.jar")))

from pyspark.ml.image import ImageSchema
from pyspark.sql.functions import lit


from pyspark.sql.functions import lit
from sparkdl.image import imageIO

cardboard_df = spark.read.format("image").load("/content/drive/MyDrive/sample/cardboard").withColumn("label", lit(0))
plastic_df = spark.read.format("image").load( "/content/drive/MyDrive/sample/glass").withColumn("label", lit(1))
paper_df = spark.read.format("image").load("/content/drive/MyDrive/sample/metal").withColumn("label", lit(2))
trash_df = spark.read.format("image").load("/content/drive/MyDrive/sample/paper").withColumn("label", lit(3))
glass_df = spark.read.format("image").load( "/content/drive/MyDrive/sample/plastic").withColumn("label", lit(4))
metal_df = spark.read.format("image").load("/content/drive/MyDrive/sample/trash").withColumn("label", lit(5))



from functools import reduce

dataframes = [cardboard_df,plastic_df, paper_df, trash_df, 
             glass_df,glass_df,metal_df]


df1 = reduce(lambda first, second: first.union(second), dataframes)

# repartition dataframe 
df1 = df1.repartition(200)


# # On hot encoding 
# from pyspark.ml.feature import OneHotEncoderEstimator
# encoder = OneHotEncoderEstimator(inputCols=["label"],outputCols=["one_hot_label"])
# model = encoder.fit(df)
# df = model.transform(df)

# split the data-frame
train, test = df1.randomSplit([0.6, 0.4], 42)

In [18]:
train.toPandas().head()

,image,label
0,(file:///content/drive/MyDrive/sample/plastic/...,4
1,(file:///content/drive/MyDrive/sample/plastic/...,4
2,(file:///content/drive/MyDrive/sample/plastic/...,4
3,(file:///content/drive/MyDrive/sample/glass/gl...,1
4,(file:///content/drive/MyDrive/sample/glass/gl...,1


In [19]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from sparkdl import DeepImageFeaturizer 

# model: InceptionV3
# extracting feature from images
featurizer = DeepImageFeaturizer(inputCol="image", outputCol="features",

                                 modelName="InceptionV3")

# used as a multi class classifier
lr = LogisticRegression(maxIter=5, regParam=0.03, 
                        elasticNetParam=0.5, labelCol="label") 

# define a pipeline model
sparkdn = Pipeline(stages=[featurizer, lr])


In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
import sparkdl
from sparkdl import DeepImageFeaturizer 

# model: InceptionV3
# extracting feature from images
featurizer = DeepImageFeaturizer(inputCol="image", outputCol="features",modelName="InceptionV3")

# used as a multi class classifier
lr = LogisticRegression(maxIter=5, regParam=0.03, 
                        elasticNetParam=0.5, labelCol="label") 

# define a pipeline model
sparkdn = Pipeline(stages=[featurizer, lr])

spark_model = sparkdn.fit(train)

In [ ]:

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# evaluate the model with test set
evaluator = MulticlassClassificationEvaluator() 
transform_test = spark_model.transform(test)

In [ ]:
print('F1-Score ', evaluator.evaluate(transform_test, 
                                      {evaluator.metricName: 'f1'}))
print('Precision ', evaluator.evaluate(transform_test,
                                       {evaluator.metricName: 'weightedPrecision'}))
print('Recall ', evaluator.evaluate(transform_test, 
                                    {evaluator.metricName: 'weightedRecall'}))
print('Accuracy ', evaluator.evaluate(transform_test, 
                                      {evaluator.metricName: 'accuracy'}))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.GnBu):

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
'''
- Convert Spark-DataFrame to Pnadas-DataFrame
- Call Confusion Matrix With 'True' and 'Predicted' Label
'''


from sklearn.metrics import confusion_matrix
y_true = transform_test.select("label")
y_true = y_true.toPandas() # convert to pandas dataframe from spark dataframe

y_pred = transform_test.select("prediction")
y_pred = y_pred.toPandas() # convert to pandas dataframe from spark dataframe

cnf_matrix = confusion_matrix(y_true, y_pred,labels=range(6))

In [ ]:
'''
- Visualize the 'Confusion Matrix' 
'''
import seaborn as sns
%matplotlib inline

sns.set_style("darkgrid")
plt.figure(figsize=(7,7))
plt.grid(False)

# call pre defined function
plot_confusion_matrix(cnf_matrix, classes=range(6))

In [ ]:
'''
- Classification Report of each class group
'''

from sklearn.metrics import classification_report

target_names = ["Class {}".format(i) for i in range(6)]
print(classification_report(y_true, y_pred, target_names = target_names))

In [ ]:

'''
- A custom ROC AUC score function for multi-class classification problem
'''

from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.preprocessing import LabelBinarizer

def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)
    return roc_auc_score(y_test, y_pred, average=average)


print('ROC AUC score:', multiclass_roc_auc_score(y_true,y_pred))

In [ ]:

'''
- Comparing true vs predicted samples
'''

# all columns after transformations
print(transform_test.columns)

# see some predicted output
transform_test.select('image', "prediction", "label").show()

In [ ]:
!pip install bigdl==0.3.0 

     |████████████████████████████████| 59.4MB 1.2MB/s 


In [ ]:
! pip install utils

In [ ]:
# Parameters
training_epochs = 10
batch_size = 128
display_step = 1

# Network Parameters
n_hidden = 32
n_input = 784 # MNIST data input (img shape: 28*28)

In [ ]:
# Create Model

def build_autoencoder(n_input, n_hidden):
    # Initialize a sequential container
    model = Sequential()

    # encoder
    model.add(Linear(n_input, n_hidden))
    model.add(ReLU())
    # decoder
    model.add(Linear(n_hidden, n_input))
    model.add(Sigmoid())
    
    return model
model = build_autoencoder(n_input, n_hidden)